In [159]:
# phone numbers and emails are interchanged
# we receive stop words in skills

In [160]:
import pandas as pd

In [161]:
df = pd.read_csv("dataset.csv")
df.head()

,Employee ID,First Name,Middle Name,Last Name,Date of Birth,Gender,Email Address,Phone Number,Address,City,...,Hire Date,Termination Date,Salary,Hourly Rate,Contract Type,Contract Start Date,Contract End Date,Skills,Certifications,Education
0,a14de57c-5e3c-4b8d-b7e5-bc2095cb1713,Connie Kathleen,NaN,Cox,1977-10-13,Female,joe16@example.net,+1-638-201-7158,"1894 Reeves Walks\nPhillipschester, CT 35434",East Amandachester,...,2020-05-22,NaN,61958,65,Full-time,2023-10-02,2024-11-02,"['lot', 'easy', 'continue']",AWS,PhD
1,de40f330-d87e-4b73-9923-b279dd7f4f19,Shane Heidi,NaN,Delgado,1966-02-20,Other,bwyatt@example.com,001-242-739-4548x043,"2178 Smith Shoals\nSouth Carolinechester, MD 1...",South Kristinstad,...,2023-09-12,2023-11-13,92575,66,Full-time,2022-12-01,2024-06-27,"['director', 'activity', 'color']",AWS,Bachelor
2,0020d213-781f-45a4-94d8-a56c914f0322,Shane Brittany,NaN,Beck,1960-12-31,Female,rjackson@example.net,(345)292-5939x93954,"7695 Matthew Roads\nJohnsonville, LA 50052",Vincentville,...,2022-03-03,NaN,38000,96,Full-time,2023-08-28,2024-05-06,"['through', 'western', 'of']",Cisco Certified,Bachelor
3,bdcac170-b7b5-4ae3-ac8b-dc1f9f32a48d,Jessica Sara,NaN,Goodman,1982-12-11,Male,(951)801-7000x845,kelsey43@example.org,"33817 Richardson Cliffs\nMatthewhaven, ME 15922",West Georgeview,...,2023-08-28,NaN,54601,33,Part-time,2021-05-22,2024-09-11,"['which', 'tax', 'society']",AWS,PhD
4,7de57982-d4da-42a6-9f72-6593cfcaa1a7,Ryan Isaiah,NaN,Kelly,1966-06-11,Other,halescott@example.org,(540)862-5111,"64245 Bush Viaduct Suite 831\nMelissafort, WA ...",Markborough,...,2023-06-11,NaN,40567,36,Contract,2021-03-06,2025-09-18,"['chair', 'eight', 'stop']",AWS,Master


### Cleaning
0. add a column for serial number
1. Check for duplicates in employee id column
2. Create a new column indicating if he is still an employee
3. Rename the columns appropriately
4. Organise the first middle and last names
5. Check if contract start and end date are corrently entered as per timeline
6. change the hourly rate and salary to range values (categorical columns)
7. move zip codes from state to zip code column
8. convert zip code into string
9. Organise phone numbers and email to their respective columns
10. Unify the format of phone numbers
11. Remove stop words from skills

Note: For every transformation, Make a note of number of rows effected

In [162]:
new_column_names = {
    'Employee ID': 'empID',
    'First Name': 'firstName',
    'Middle Name': 'middleName',
    'Last Name': 'lastName',
    'Date of Birth': 'dateOfBirth',
    'Gender': 'gender',
    'Email Address': 'emailAddress',
    'Phone Number': 'phoneNumber',
    'Address': 'address',
    'City': 'city',
    'State/Province': 'state',
    'Zip Code/Postal Code': 'postalCode',
    'Country': 'country',
    'Job Title': 'jobTitle',
    'Department': 'department',
    'Manager ID': 'managerID',
    'Hire Date': 'hireDate',
    'Termination Date': 'terminationDate',
    'Salary': 'salary',
    'Hourly Rate': 'hourlyRate',
    'Contract Type': 'contractType',
    'Contract Start Date': 'contractStartDate',
    'Contract End Date': 'contractEndDate',
    'Skills': 'skills',
    'Certifications': 'certifications',
    'Education': 'education'
}

In [163]:
# Utility functions
import re

def extract_phone_number(num):
    num = re.sub(r'[^0-9\s]+', '', num)
    formatted_number = f"({num[:3]}) {num[3:6]}-{num[6:10]}"
    return formatted_number

def round_figure(num):
    first_starting = str(num)[0]
    second_starting = str(num)[0]

    first_ending = "0" * (len(str(num))-1)
    second_ending = "9" * (len(str(num))-1)
    
    return f"{first_starting}{first_ending} - {second_starting}{second_ending}"

In [164]:
def rename_columns(df : pd.DataFrame) -> pd.DataFrame :
    df.rename(columns=new_column_names, inplace=True)
def to_catagorical(df : pd.DataFrame, column_names : list) -> pd.DataFrame :
    salary_col, rate_col = column_names
    df['salaryRange'] = df[salary_col].map(lambda x: round_figure(x))
    df['hourlyRateRange'] = df[rate_col].map(lambda x: round_figure(x))

# Add new columns
def add_serial_number_column(df : pd.DataFrame) -> pd.DataFrame :
    df["serialNo"] = pd.Series(range(len(df)))
def add_isEmployee_column(df : pd.DataFrame, column_names : list) -> pd.DataFrame :
    col = column_names[0]
    df['isEmployee'] = df[col].isnull()

# Validate existing columns
def validate_employee_id_column(df : pd.DataFrame, column_names : list) -> pd.DataFrame :
    col = column_names[0]
    has_duplicates = not all(df.drop_duplicates(subset = [col])[col] == df[col])
    if has_duplicates:
        print(f"Dataframe has duplicates in {col}")
    else:
        print(f"No duplicates found in {col}")
def validate_start_end_date_columns(df : pd.DataFrame, column_names : list) -> pd.DataFrame :
    start_col, end_col = column_names
    df[start_col] = pd.to_datetime(df[start_col], format='%Y-%m-%d')
    df[end_col] = pd.to_datetime(df[end_col], format='%Y-%m-%d')
    df['valid_dates'] = df[start_col] <= df[end_col]
    if all(df['valid_dates']):
        print(f"All dates are valid")
    else:
        n = df['valid_dates'].value_counts()[False]
        print(f"There are {n} rows with invalid dates")
    df.drop(columns=['valid_dates'], inplace=True)
def validate_skills_column(df : pd.DataFrame, column_names : list) -> pd.DataFrame :
    pass

# organise functions
def organise_state_zip_columns(df : pd.DataFrame, column_names : list) -> pd.DataFrame :
    state_col, zip_col = column_names
    for index, row in df.iterrows():
        vals = row[state_col].split(" ")
        try:
            zip = int(vals[-1])
            state = " ".join(vals[:-1])
        except:
            state = row[state_col]
            zip = row[zip_col]
        df.loc[index, state_col] = state
        df.loc[index, zip_col] = zip
def organise_employee_name_columns(df : pd.DataFrame, column_names : list) -> pd.DataFrame :
    fn_col, mn_col, ln_col = column_names

    df['first_name_1'] = df[fn_col].fillna("")
    df['middle_name_1'] = df[mn_col].fillna("")
    df['last_name_1'] = df[ln_col].fillna("")

    for index, row in df.iterrows():
        full_name = row['first_name_1'] + " " + row['middle_name_1'] + " " + row['last_name_1']
        full_name = full_name.split(" ")
        first_name, middle_name, last_name = full_name[0], " ".join(full_name[1:-2]), full_name[-1]
        df.loc[index, fn_col] = first_name
        df.loc[index, mn_col] = middle_name
        df.loc[index, ln_col] = last_name
    df.drop(columns=['first_name_1', 'middle_name_1', 'last_name_1'], inplace=True)
def organise_email_phone_columns(df : pd.DataFrame, column_names : list) -> pd.DataFrame :
    phone_col, email_col = column_names
    for index, row in df.iterrows():
        if "@" in row[phone_col]:
            temp_phone = row[phone_col]
            temp_email = row[email_col]
            df.loc[index, phone_col] = temp_email
            df.loc[index, email_col] = temp_phone

# reformat functions
def reformat_zip_code_column(df : pd.DataFrame, column_names : list) -> pd.DataFrame :
    zip_col = column_names[0]
    data = df[zip_col].astype(str)
    df.drop(columns=[zip_col], axis=0, inplace=True)
    df[zip_col] = data

def reformat_phone_numbers(df : pd.DataFrame, column_names : list) -> pd.DataFrame :
    phone_col = column_names[0]
    df[phone_col] = df[phone_col].map(lambda x: extract_phone_number(x))

In [165]:
to_catagorical(df = df, column_names = ['Salary', 'Hourly Rate'])
add_serial_number_column(df = df)
add_isEmployee_column(df = df, column_names = ['Termination Date'])
validate_employee_id_column(df = df, column_names = ['Employee ID'])
validate_start_end_date_columns(df = df, column_names = ['Contract Start Date', 'Contract End Date'])
validate_skills_column(df = df, column_names = [])
reformat_zip_code_column(df = df, column_names = ['Zip Code/Postal Code'])
organise_state_zip_columns(df = df, column_names = ['State/Province', 'Zip Code/Postal Code'])
organise_employee_name_columns(df = df, column_names = ['First Name', 'Middle Name', 'Last Name'])
organise_email_phone_columns(df = df, column_names = ['Email Address', 'Phone Number'])
reformat_phone_numbers(df = df, column_names = ['Phone Number'])
rename_columns(df = df)

No duplicates found in Employee ID
All dates are valid


In [167]:
df.to_csv("cleaned_dataset.csv", index=False)